# Assingment 4 Testing and Stuff

$H(s)= - \sum_{i, j} J_{i, j} * s_i * s_j$

First step: load in data and interpret '+' and '-' as +1 and -1

In [30]:
import numpy as np
import os

cwd = os.getcwd()
ass_dir = cwd.rsplit('\\', maxsplit=1)[0]

#import
pm_data_str = np.loadtxt(os.path.join(ass_dir, r'data\in.txt'), dtype=str)

# separate string into characters
pm_data_sep = np.empty((pm_data_str.shape[0], len(pm_data_str[0])), dtype=str)
for i in range(len(pm_data_str)):
    pm_data_sep[i] = list(pm_data_str[i])

# locations of plus minus
p_loc = np.where(pm_data_sep == '+')
m_loc = np.where(pm_data_sep == '-')

# convert plus/minus to +1/-1
pm_data = np.empty(pm_data_sep.shape)
pm_data[p_loc] = 1.
pm_data[m_loc] = -1.
pm_data

array([[-1.,  1., -1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1., -1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1.,  1.],
       [ 1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.]])

In [26]:
len(pm_data_str[0])
pm_data_str.shape

(1000,)